### Determining the optimal number of hidden layers and neurons for an Artificial Neural Network (ANN) 
This can be challenging and often requires experimentation. However, there are some guidelines and methods that can help you in making an informed decision:

- Start Simple: Begin with a simple architecture and gradually increase complexity if needed.
- Grid Search/Random Search: Use grid search or random search to try different architectures.
- Cross-Validation: Use cross-validation to evaluate the performance of different architectures.
- Heuristics and Rules of Thumb: Some heuristics and empirical rules can provide starting points, such as:
  -    The number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer.
  -  A common practice is to start with 1-2 hidden layers.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [4]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
print("GPUs found:", gpus)


GPUs found: []


In [5]:
# Comprehensive GPU Detection and Setup
import tensorflow as tf
import platform
import subprocess
import sys

print("=== System Information ===")
print(f"Python version: {sys.version}")
print(f"TensorFlow version: {tf.__version__}")
print(f"Platform: {platform.platform()}")

print("\n=== GPU Detection ===")
gpus = tf.config.list_physical_devices('GPU')
print(f"Number of GPUs found: {len(gpus)}")
for i, gpu in enumerate(gpus):
    print(f"GPU {i}: {gpu}")

print("\n=== CUDA Information ===")
print(f"CUDA available: {tf.test.is_built_with_cuda()}")
print(f"GPU available: {tf.test.is_gpu_available()}")

# Check if CUDA is properly installed
try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True, timeout=10)
    if result.returncode == 0:
        print("\n=== NVIDIA-SMI Output ===")
        print(result.stdout)
    else:
        print("nvidia-smi command failed")
except (subprocess.TimeoutExpired, FileNotFoundError):
    print("nvidia-smi not found or timed out")

print("\n=== Recommendations ===")
if len(gpus) == 0:
    print("No GPUs detected. To use your RTX 4060:")
    print("1. Install CUDA 11.8 or 12.x from NVIDIA")
    print("2. Install cuDNN 8.x")
    print("3. Install tensorflow-gpu or tensorflow[and-cuda]")
    print("4. Restart your Python environment")
    print("\nCommands to install GPU TensorFlow:")
    print("pip uninstall tensorflow")
    print("pip install tensorflow[and-cuda]")

=== System Information ===
Python version: 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]
TensorFlow version: 2.20.0
Platform: Windows-11-10.0.26100-SP0

=== GPU Detection ===
Number of GPUs found: 0

=== CUDA Information ===
CUDA available: False
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU available: False

=== NVIDIA-SMI Output ===
Thu Aug 28 12:33:42 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 581.08                 Driver Version: 581.08         CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |              

In [6]:
# Install GPU-enabled TensorFlow with improved error handling
import subprocess
import sys

try:
    print("Step 1: Upgrading pip...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "pip"])
    
    print("Step 2: Installing TensorFlow with GPU support...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "tensorflow[and-cuda]"])
    
    print("Installation complete! Please restart the kernel to use GPU support.")
    
except subprocess.CalledProcessError as e:
    print(f"Installation failed with error: {e}")
    print("Try running the commands manually in terminal:")
    print("1. python -m pip install --upgrade pip")
    print("2. python -m pip install tensorflow[and-cuda]")
    
except Exception as e:
    print(f"Unexpected error: {e}")
    print("Please try manual installation in terminal.")

Installing GPU-enabled TensorFlow...
This will uninstall the current TensorFlow and install the GPU version.


CalledProcessError: Command '['d:\\Deep Learning\\ann_env_new\\Scripts\\python.exe', '-m', 'pip', 'install', 'tensorflow[and-cuda]']' returned non-zero exit status 1.

In [4]:
data=pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [5]:
## Define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model



In [6]:
## Create a Keras classifier
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,verbose=1)

In [7]:

# Define the grid search parameters
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [8]:
# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=1)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits


d:\Deep Learning\ann_env_new\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
d:\Deep Learning\ann_env_new\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7946 - loss: 0.4994
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8125 - loss: 0.4330
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8256 - loss: 0.4083
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8375 - loss: 0.3879
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8478 - loss: 0.3693
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8547 - loss: 0.3577
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8555 - loss: 0.3513
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8576 - loss: 0.3475
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8591 - loss: 0.3443
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8610 - loss: 0.3418
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8608 - loss: 0.3408
Epoch 12/100
250/250 ━━━━━━━━━━━━━━━━━━━━